# VL-T5 inference on custom images

## Download code and install dependencies

In [1]:
!git clone https://github.com/j-min/VL-T5


Cloning into 'VL-T5'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (135/135), done.
remote: Total 229 (delta 132), reused 177 (delta 89), pack-reused 0
Receiving objects: 100% (229/229), 900.48 KiB | 8.11 MiB/s, done.
Resolving deltas: 100% (132/132), done.


In [2]:
cd VL-T5

/content/VL-T5


In [3]:
!pip uninstall param -y # to resolve name conflict with src.param.py
!pip install -r requirements.txt
!python download_backbones.py

Found existing installation: param 1.12.0
Uninstalling param-1.12.0:
  Successfully uninstalled param-1.12.0
  Cloning git://github.com/j-min/language-evaluation (to revision master) to /tmp/pip-req-build-ayagtpsy
  Running command git clone -q git://github.com/j-min/language-evaluation /tmp/pip-req-build-ayagtpsy
     |████████████████████████████████| 748.8 MB 19 kB/s 
     |████████████████████████████████| 1.8 MB 38.2 MB/s 
     |████████████████████████████████| 1.2 MB 37.1 MB/s 
     |████████████████████████████████| 1.7 MB 30.5 MB/s 
     |████████████████████████████████| 90 kB 9.4 MB/s 
     |████████████████████████████████| 5.9 MB 24.9 MB/s 
     |████████████████████████████████| 2.9 MB 32.2 MB/s 
     |████████████████████████████████| 895 kB 39.7 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |████████████████████████████████| 180 kB 39.0 MB/s 
     |████████████████████████████████| 140 kB 47.7 MB/s 
     |████████████████████████████████| 63 kB 1.7 

## Download the pretrained checkpoint

In [4]:
import gdown

In [5]:
!mkdir -p VL-T5/snap/pretrain/VLBart
!mkdir -p VL-T5/snap/pretrain/VLT5

In [6]:
gdown.download('https://drive.google.com/uc?id=100qajGncE_vc4bfjVxxICwz3dwiAxbIZ', 'VL-T5/snap/pretrain/VLT5/Epoch30.pth', quiet=False)
gdown.download('https://drive.google.com/uc?id=1fTKGCBfMe2eJ_rivPQu0YkNJTNdv_0aq', 'VL-T5/snap/pretrain/VLBart/Epoch30.pth', quiet=False)

Downloading...
From: https://drive.google.com/uc?id=100qajGncE_vc4bfjVxxICwz3dwiAxbIZ
To: /content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30.pth
100%|██████████| 898M/898M [00:07<00:00, 122MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1fTKGCBfMe2eJ_rivPQu0YkNJTNdv_0aq
To: /content/VL-T5/VL-T5/snap/pretrain/VLBart/Epoch30.pth
100%|██████████| 565M/565M [00:16<00:00, 34.9MB/s]


'VL-T5/snap/pretrain/VLBart/Epoch30.pth'

Download datasets

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive/MyDrive/datasets

!cp -r '/content/gdrive/MyDrive/datasets' '/content/VL-T5'
!ls /content/VL-T5/datasets

Mounted at /content/gdrive
COCO  flickr30k  GQA  lxmert  nlvr  RefCOCO  VCR  VG  vqa
COCO  flickr30k  GQA  lxmert  nlvr  RefCOCO  VCR  VG  vqa


In [8]:
import sys
sys.path.append('./VL-T5/src')

In [9]:
from param import parse_args
args = parse_args(
    backbone='t5-base', # Backbone architecture
    load='/content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30', # Pretrained checkpoint
    parse=False, # False for interactive env (ex. jupyter)'
    log_train_accuracy=True,
    gpu=0
)

In [10]:
!ls /content/VL-T5/VL-T5/snap/pretrain/VLT5

Epoch30.pth


In [11]:
# Load data loaders
from vcr_data import get_loader
train_loader = get_loader(
    args,
    split='train'
)


Data sources:  ['train']


Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Loaded 212923 data from train
# all sentences: 212923


In [12]:
val_loader = get_loader(
    args,
    split='val',
)



Data sources:  ['val']
Loaded 26534 data from val
# all sentences: 26534


In [13]:
test_loader = get_loader(
    args,
    split='test',
)

Data sources:  ['test']
Loaded 25263 data from test
# all sentences: 25263


In [14]:
# Import trainer
from vcr import Trainer
trainer = Trainer(
    args,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
)

# model is attached to trainer
model = trainer.model

Building Model at GPU 0
Model loaded from  /content/VL-T5/VL-T5/snap/pretrain/VLT5/Epoch30.pth
_IncompatibleKeys(missing_keys=[], unexpected_keys=['encoder.visual_embedding.layer_norm.weight'])
Model Launching at GPU 0
Building Optimizer
Batch per epoch: 6654
Total Iters: 79848
Warmup ratio: 0.05
Warm up Iters: 3992
It took 5.1s


In [15]:
train_batch = next(iter(train_loader))


In [16]:
train_batch['log_train_accuracy'] = True


In [17]:
model.train_step(train_batch)

RuntimeError: ignored

In [ ]:
# Inference
test_batch = next(iter(test_loader))


In [ ]:
model.test_step(test_batch)

In [ ]:
test_batch